In [ ]:
%pip install pandas matplotlib mysqlclient sqlalchemy

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt

# 替换以下字符串为你的数据库连接信息
# 格式：'mysql+pymysql://用户名:密码@主机名:端口/数据库名'
# HOST = "gateway01.us-west-2.prod.aws.tidbcloud.com"
# PORT = 4000
# USR = "4JHciBfpftZR67E.root"
# PWD = "aidBsAjcm2YUaFac"
# DB = "data"
TABLE = "hk_immigration_purge"
# database_url = f"mysql+mysqldb://{USR}:{PWD}@{HOST}:{PORT}/{DB}"
database_url = 'sqlite:///../data/data.sqlite'

# 创建数据库引擎
engine = create_engine(database_url)

In [13]:
query2020 = f"SELECT * FROM {TABLE} WHERE 日期 >= '2020-05-01' AND 日期 <= '2020-12-31';"
df2020 = pd.read_sql_query(query2020, engine)

query2021 = f"SELECT * FROM {TABLE} WHERE 日期 >= '2021-01-01' AND 日期 <= '2021-12-31';"
df2021 = pd.read_sql_query(query2021, engine)

query2022 = f"SELECT * FROM {TABLE} WHERE 日期 >= '2022-01-01' AND 日期 <= '2022-12-31';"
df2022 = pd.read_sql_query(query2022, engine)

query2023 = f"SELECT * FROM {TABLE} WHERE 日期 >= '2023-01-01' AND 日期 <= '2023-12-31';"
df2023 = pd.read_sql_query(query2023, engine)

In [14]:
from matplotlib import font_manager
# fname中选择一个你本机查询出来的字体 若没有中文字体则需要你本人手动安装
plt.rc("font",family='MicroSoft YaHei',weight="bold")
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像时负号'-'显示为方块的问题

In [15]:
import matplotlib.pyplot as plt
import pandas as pd

# 假设df2020, df2021, df2022, df2023已经被正确加载和预处理
# 将日期列转换为日期时间格式
df2020['日期'] = pd.to_datetime(df2020['日期'])
df2021['日期'] = pd.to_datetime(df2021['日期'])
df2022['日期'] = pd.to_datetime(df2022['日期'])
df2023['日期'] = pd.to_datetime(df2023['日期'])

# 设置图形大小并创建一个 2x2 的图表布局
fig, axs = plt.subplots(2, 2, figsize=(20, 12))

# 定义一个小函数来简化代码
def plot_data(ax, df, title):
    for column in df.columns:
        if column != '日期':
            ax.plot(df['日期'], df[column], label=column)
    ax.legend()
    ax.set_xlabel('Date')
    ax.set_ylabel('Value')
    ax.set_title(title)

# 分别绘制每年的数据
plot_data(axs[0, 0], df2020, 'Data Visualization 2020')
plot_data(axs[0, 1], df2021, 'Data Visualization 2021')
plot_data(axs[1, 0], df2022, 'Data Visualization 2022')
plot_data(axs[1, 1], df2023, 'Data Visualization 2023')

# 调整子图间距
plt.tight_layout()
plt.show()


In [16]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd

# 假设df2020, df2021, df2022, df2023已经被正确加载和预处理
# 将日期列转换为日期时间格式
df2020['日期'] = pd.to_datetime(df2020['日期'])
df2021['日期'] = pd.to_datetime(df2021['日期'])
df2022['日期'] = pd.to_datetime(df2022['日期'])
df2023['日期'] = pd.to_datetime(df2023['日期'])

# 创建归一化器实例
scaler = MinMaxScaler()

# 定义一个函数来归一化除了日期外的所有列
def normalize_df(df):
    date_column = df['日期']
    df_scaled = pd.DataFrame(scaler.fit_transform(df.drop('日期', axis=1)), columns=df.columns[1:])
    df_scaled['日期'] = date_column
    return df_scaled

# 归一化数据
df2020_normalized = normalize_df(df2020)
df2021_normalized = normalize_df(df2021)
df2022_normalized = normalize_df(df2022)
df2023_normalized = normalize_df(df2023)

# 绘图代码，使用归一化后的数据
fig, axs = plt.subplots(2, 2, figsize=(20, 12))

# 定义一个小函数来简化代码
def plot_data(ax, df, title):
    for column in df.columns:
        if column != '日期':
            ax.plot(df['日期'], df[column], label=column)
    ax.legend()
    ax.set_xlabel('Date')
    ax.set_ylabel('Value')
    ax.set_title(title)

# 分别绘制每年的数据
plot_data(axs[0, 0], df2020_normalized, 'Data Visualization 2020')
plot_data(axs[0, 1], df2021_normalized, 'Data Visualization 2021')
plot_data(axs[1, 0], df2022_normalized, 'Data Visualization 2022')
plot_data(axs[1, 1], df2023_normalized, 'Data Visualization 2023')

# 调整子图间距
plt.tight_layout()
plt.show()

In [ ]:
import math
import pandas as pd
from dtaidistance import dtw
import seaborn as sns
import matplotlib.pyplot as plt

# 假设 df 是你的DataFrame，并且所有列都是数值型数据
df = df2023.drop(columns=['日期'])

# 创建一个空的DataFrame用于存储DTW距离
columns = df.columns
distance_matrix = pd.DataFrame(index=columns, columns=columns, data=0.0)

# 计算DTW距离并填充距离矩阵
for col1 in df.columns:
    for col2 in df.columns:
        if col1 != col2:
            distance = math.log(dtw.distance(df[col1].values, df[col2].values))
            distance_matrix.loc[col1, col2] = distance
        else:
            # 对角线上的距离设置为0（同一个列与自己的距离）
            distance_matrix.loc[col1, col2] = 0

# 使用Seaborn绘制热图
plt.figure(figsize=(10, 8))
sns.heatmap(distance_matrix, annot=True, cmap="viridis")
plt.title("DTW Distance Matrix")
plt.show()
